In [1]:
from ds_loader import Dataset_Handler
from torchvision.models.mobilenet import mobilenet_v2
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
'''
Initialise some things
'''
class Config():
    batch_size = 32
    epochs = 50
    learn_rate = 0.0001
    CLASSES = 5
    generate = False
dsh = Dataset_Handler(dataset_folder='sleep_lab_data', target_hertz=50)

In [3]:
'''
Generate the data and store localy
'''
if Config.generate: dsh.generate_dataset()

In [8]:
'''
Load Data(numpy array) from disk)
'''
train_dl = dsh.get_dataloader(herz=50,modality="baseline.npy", batch_size=32)

In [9]:
'''
take fist network architecture
'''
# check for device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# define model
model = mobilenet_v2(pretrained=True)
model.features[0][0] = nn.Conv2d(11, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=Config.CLASSES)
model.to(device)

# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=Config.learn_rate)
# define loss function
loss_function = nn.CrossEntropyLoss()

# check if model is on correct device
next(model.parameters()).is_cuda



False

In [ ]:
# Traning Process
loss_ls_train = []
model.train()

for epoch in range(Config.epochs):
    for data, labels in train_dl:
        data, labels = data.to(device), labels.to(device)


        optimizer.zero_grad()
        data = data.permute(0, 2, 1)
        data = data.unsqueeze(2)

        prediction = model(data)

        loss = loss_function(prediction, labels)
        loss.backward()
        optimizer.step()

        loss_ls_train.append(loss.data.item())

    print('===> Epoch: {} loss: {:.4f}'.format(epoch, loss.data.item()))

plt.plot(loss_ls_train)
plt.show()
